In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [6]:
data = pd.read_excel("../data/Export_TBL_MVA.xlsx")
data = data[(data["GELOESCHT"] != 1) & (data["STATUS"] == "TA erstellt")]
package_data = pd.read_excel("../data/Export_Packstuecke_MVA.xlsx")
df_merged = pd.merge(
    data, package_data, left_on="ID", right_on="TBL_MVA_ID", how="inner"
)

In [7]:
df_merged.isnull().sum()

ID_x                               0
AUFTRAGSNUMMER                     0
EINGANGSDATUM_UHRZEIT              0
LIEFERSCHEINNUMMER                 0
STATUS                             0
VERPACKT_DATUM_UHRZEIT             0
SONDERFAHRT                        0
LIEFERSCHEIN_DATUM_UHRZEIT         0
BEREITGESTELLT_DATUM_UHRZEIT       0
TA_DATUM_UHRZEIT                   0
NOTIZ                              0
AUFTRAGANNAHME_DATUM_UHRZEIT       0
DISPLAYNAME                        0
LAND                               0
GELOESCHT                          0
BEMERKUNG_GELOESCHT                0
ARCHIV                             0
GELOESCHT_DATUM_UHRZEIT            0
TRACKING_NUMMER                  702
DIENSTLEISTER                    702
KLAERUNG_LS                      912
KLAERUNG_TA                      912
GEFAHRGUT                       3026
PO_NUMMER                       3026
ANGEKUENDIGT                    3026
PRIO                            3026
VERSENDER                       3026
I

In [8]:
df_merged = df_merged.drop(
    columns=[
        "ID_x",
        "AUFTRAGSNUMMER",
        "LIEFERSCHEINNUMMER",
        "NOTIZ",
        "DISPLAYNAME",
        "GELOESCHT",
        "BEMERKUNG_GELOESCHT",
        "ARCHIV",
        "GELOESCHT_DATUM_UHRZEIT",
        "TRACKING_NUMMER",
        "KLAERUNG_LS",
        "KLAERUNG_TA",
        "GEFAHRGUT",
        "PO_NUMMER",
        "ANGEKUENDIGT",
        "VERSENDER",
        "ID_y",
        "PACKSTUECK_ID",
        "TBL_MVA_ID",
    ]
)

In [9]:
df_merged["PACKSTUECKART"].unique()
mode_value = df_merged["PACKSTUECKART"].mode()[0]
df_merged["PACKSTUECKART"] = df_merged["PACKSTUECKART"].replace(" ", mode_value)
label_encoder = LabelEncoder()
df_merged["PACKSTUECKART"] = label_encoder.fit_transform(df_merged["PACKSTUECKART"])

# package details
df_merged["PACKAGE_VOLUME"] = (
    df_merged["LAENGE_IN_CM"] * df_merged["BREITE_IN_CM"] * df_merged["HOEHE_IN_CM"]
)
df_merged["PACKAGE_DENSITY"] = df_merged["GEWICHT_IN_KG"] / df_merged["PACKAGE_VOLUME"]
mean_package_density = df_merged["PACKAGE_DENSITY"].mean()
df_merged["PACKAGE_DENSITY"].fillna(mean_package_density, inplace=True)

df_merged["PACKSTUECKART"].unique()
mode_value = df_merged["PACKSTUECKART"].mode()[0]
df_merged["PACKSTUECKART"] = df_merged["PACKSTUECKART"].replace(" ", mode_value)
label_encoder = LabelEncoder()
df_merged["PACKSTUECKART"] = label_encoder.fit_transform(df_merged["PACKSTUECKART"])


# time difference
df_merged["PROCESSING_TIME_DURATION"] = (
    pd.to_datetime(df_merged["BEREITGESTELLT_DATUM_UHRZEIT"])
    - pd.to_datetime(df_merged["EINGANGSDATUM_UHRZEIT"])
).dt.total_seconds() / 3600  # in hours
df_merged["TIME_DIFF_VERPACKT_TO_LIEFERSCHEIN"] = (
    pd.to_datetime(df_merged["LIEFERSCHEIN_DATUM_UHRZEIT"])
    - pd.to_datetime(df_merged["VERPACKT_DATUM_UHRZEIT"])
).dt.total_seconds() / 3600  # in hours
df_merged["TIME_DIFF_LIEFERSCHEIN_TO_BEREITGESTELLT"] = (
    pd.to_datetime(df_merged["BEREITGESTELLT_DATUM_UHRZEIT"])
    - pd.to_datetime(df_merged["LIEFERSCHEIN_DATUM_UHRZEIT"])
).dt.total_seconds() / 3600  # in hours
df_merged["TIME_DIFF_BEREITGESTELLT_TO_TA"] = (
    pd.to_datetime(df_merged["TA_DATUM_UHRZEIT"])
    - pd.to_datetime(df_merged["BEREITGESTELLT_DATUM_UHRZEIT"])
).dt.total_seconds() / 3600  # in hours
# df_merged["SPECIAL_HANDLING"] = df_merged["SONDERFAHRT"].astype(int)


# specific time
df_merged["HOUR_OF_DAY_EINGANG"] = pd.to_datetime(
    df_merged["EINGANGSDATUM_UHRZEIT"]
).dt.hour
df_merged["HOUR_OF_DAY_VERPACKT"] = pd.to_datetime(
    df_merged["VERPACKT_DATUM_UHRZEIT"]
).dt.hour
df_merged["HOUR_OF_DAY_LIEFERSCHEIN"] = pd.to_datetime(
    df_merged["LIEFERSCHEIN_DATUM_UHRZEIT"]
).dt.hour
df_merged["HOUR_OF_DAY_BEREITGESTELLT"] = pd.to_datetime(
    df_merged["BEREITGESTELLT_DATUM_UHRZEIT"]
).dt.hour
df_merged["HOUR_OF_DAY_TA"] = pd.to_datetime(df_merged["TA_DATUM_UHRZEIT"]).dt.hour

# historical data
df_merged["HISTORICAL_PROCESSING_TIME_MEAN"] = df_merged.groupby("PACKSTUECKART")[
    "PROCESSING_TIME_DURATION"
].transform("mean")
# df_merged["HISTORICAL_PROCESSING_TIME_MEDIAN"] = df_merged.groupby("PACKSTUECKART")[
#     "PROCESSING_TIME_DURATION"
# ].transform("median")
df_merged["HISTORICAL_PROCESSING_TIME_STD"] = df_merged.groupby("PACKSTUECKART")[
    "PROCESSING_TIME_DURATION"
].transform("std")

# df_merged = df_merged.drop(
#     columns=[
#         "Shipping_order_number",
#         "TBL_MVA_ID",
#         # "PACKSTUECKART",
#         "PACKSTUECK_ID",
#         "ID_y",
#         "ID_x",
#         "LAENGE_IN_CM",
#         "BREITE_IN_CM",
#         "HOEHE_IN_CM",
#     ]
# )
# df_merged

most_common_provider = df_merged["DIENSTLEISTER"].mode()[0]
df_merged["DIENSTLEISTER"] = df_merged["DIENSTLEISTER"].replace(
    " ", most_common_provider
)
df_merged["DIENSTLEISTER"] = df_merged["DIENSTLEISTER"].replace(
    "0", most_common_provider
)
df_merged["DIENSTLEISTER"] = df_merged["DIENSTLEISTER"].fillna(most_common_provider)
label_encoder = LabelEncoder()
df_merged["DIENSTLEISTER"] = label_encoder.fit_transform(df_merged["DIENSTLEISTER"])

df_merged["PRIO"] = df_merged["PRIO"].fillna(1)
df_merged["PRIO"] = df_merged["PRIO"].replace(" ", 1)
df_merged["PRIO"] = df_merged["PRIO"].astype(int)

df_merged["LAND"] = df_merged["LAND"].str.strip()
df_merged["LAND"] = df_merged["LAND"].replace(
    [
        "",
        "de",
        "De",
        "DE - FCA Brucker",
        "DE - DAP | Schenker",
        "^DE",
        "DE - Schenker/UPS",
        "dE",
        "de- Koller",
        "DER",
        "TNT | 070275454 | FCA",
        "FCA",
        "D",
        "DR",
    ],
    "DE",
)
df_merged["LAND"] = df_merged["LAND"].replace(["Österreich", "at", "AUT"], "AT")
df_merged["LAND"] = df_merged["LAND"].replace(["IN - Bidadi"], "IN")
df_merged["LAND"] = df_merged["LAND"].replace(["Dänemark"], "DK")
df_merged["LAND"] = df_merged["LAND"].replace(["Tr"], "TR")

label_encoder = LabelEncoder()
df_merged["LAND"] = label_encoder.fit_transform(df_merged["LAND"])


df_merged = df_merged.drop(
    columns=[
        "EINGANGSDATUM_UHRZEIT",
        "STATUS",
        "VERPACKT_DATUM_UHRZEIT",
        "LIEFERSCHEIN_DATUM_UHRZEIT",
        "BEREITGESTELLT_DATUM_UHRZEIT",
        "TA_DATUM_UHRZEIT",
        "AUFTRAGANNAHME_DATUM_UHRZEIT",
        "PRIO",
        "LAENGE_IN_CM",
        "BREITE_IN_CM",
        "HOEHE_IN_CM",
        "GEWICHT_IN_KG",
        "SONDERFAHRT",
        "LAND",
        "DIENSTLEISTER",
        "PACKSTUECKART",
        # "TIME_DIFF_VERPACKT_TO_LIEFERSCHEIN",
        # "TIME_DIFF_LIEFERSCHEIN_TO_BEREITGESTELLT",
        # "TIME_DIFF_BEREITGESTELLT_TO_TA",
        "HISTORICAL_PROCESSING_TIME_MEAN",
        "HISTORICAL_PROCESSING_TIME_STD",
    ]
)

C:\Users\92335\AppData\Local\Temp\ipykernel_19288\2045671395.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged["PACKAGE_DENSITY"].fillna(mean_package_density, inplace=True)


In [10]:
df_merged.isnull().sum()

PACKAGE_VOLUME                              0
PACKAGE_DENSITY                             0
PROCESSING_TIME_DURATION                    0
TIME_DIFF_VERPACKT_TO_LIEFERSCHEIN          0
TIME_DIFF_LIEFERSCHEIN_TO_BEREITGESTELLT    0
TIME_DIFF_BEREITGESTELLT_TO_TA              0
HOUR_OF_DAY_EINGANG                         0
HOUR_OF_DAY_VERPACKT                        0
HOUR_OF_DAY_LIEFERSCHEIN                    0
HOUR_OF_DAY_BEREITGESTELLT                  0
HOUR_OF_DAY_TA                              0
dtype: int64

In [11]:
for column in ["PACKAGE_DENSITY"]:

    # Calculate the first quartile (Q1) and third quartile (Q3)
    Q1 = df_merged[column].quantile(0.25)
    Q3 = df_merged[column].quantile(0.75)

    # Calculate the interquartile range (IQR)
    IQR = Q3 - Q1

    # Define the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Filter the DataFrame to remove outliers
    df_merged = df_merged[
        (df_merged[column] >= lower_bound) & (df_merged[column] <= upper_bound)
    ]

    # Display the filtered data
df_merged

,PACKAGE_VOLUME,PACKAGE_DENSITY,PROCESSING_TIME_DURATION,TIME_DIFF_VERPACKT_TO_LIEFERSCHEIN,TIME_DIFF_LIEFERSCHEIN_TO_BEREITGESTELLT,TIME_DIFF_BEREITGESTELLT_TO_TA,HOUR_OF_DAY_EINGANG,HOUR_OF_DAY_VERPACKT,HOUR_OF_DAY_LIEFERSCHEIN,HOUR_OF_DAY_BEREITGESTELLT,HOUR_OF_DAY_TA
0,864000,0.000356,21.627222,1.294167,17.780278,3.851944,8,10,12,5,9
4,864000,0.000461,21.627222,1.294167,17.780278,3.851944,8,10,12,5,9
6,8000,0.000125,96.575000,71.201111,25.366667,3.804444,4,4,4,5,9
7,8000,0.000250,96.589167,71.270278,25.304444,3.910278,4,5,4,5,9
9,16576,0.000121,96.566667,71.367778,25.188056,4.819722,5,5,4,5,10
...,...,...,...,...,...,...,...,...,...,...,...
4843,19872,0.000080,68.361667,1.086111,67.269167,0.329444,9,9,10,5,6
4844,28800,0.000243,68.295556,2.655278,65.633889,1.495278,9,9,12,6,7
4845,18000,0.000289,68.343056,3.357500,64.980556,0.865833,9,9,13,6,7
4846,6480,0.000071,68.278056,3.378889,64.895556,0.815000,9,9,13,6,7


In [12]:
df_merged.to_excel("../data/pre_processed_data.xlsx", index=False)